# Part 2: Asymmetric encryption using Rivest-Shamir-Adleman (RSA)

## Task 1: <font color='gray'>How _NOT_ to do it</font>

Use the following links and the information you've already learned
**to figure out the potential issues** in the next `code` cell
that uses RSA API. The code itself might not even be runable.

- [`cryptography.rsa`](https://cryptography.io/en/latest/hazmat/primitives/asymmetric/rsa/?highlight=rsa#) module documentation
- [`PKCS1v15 padding caveats`](https://cryptography.io/en/latest/limitations/?highlight=PKCS1%20v1.5#rsa-pkcs1-v1-5-constant-time-decryption)
- [Cryptography right answers](https://www.daemonology.net/blog/2009-06-11-cryptographic-right-answers.html)

In [ ]:
from cryptography.hazmat.primitives.asymmetric import rsa, padding

# Hint: What are the reasonable values for RSA keypair generation?
private_key = rsa.generate_private_key(
    public_exponent=3,
    key_size=512,
)

message = b"message"
# obtain the public key from the private key
public_key = private_key.public_key()

# Hint: What options do we have for the padding?
pad = padding.PKCS1v15()

# Hint: What if we encrypt messages of a different length?
ciphertext = public_key.encrypt(
    message,
    pad,
)

In [ ]:
private_key = rsa.generate_private_key(
    # Issue #1: `65537` is recommended as the public exponent
    public_exponent=3,
    # Issue #2: keys shorter then 2048 bits are not generally considered secure
    key_size=512,
)

public_key = private_key.public_key()
# Issue #3: PKCS1v15 is not the recommended padding, especially in the
# combination with public exponent `3`
pad = padding.PKCS1v15()
ciphertext = public_key.encrypt(
    # Issue #4: encrypting messages longer than the modulus is not possible
    b"message" * 100,
    pad,
)

# More on:
# combination of `PKCS1v15` and `public_exponent=3` is known to be
# vulnerable to some attacks (search for PKCS):
# https://www.daemonology.net/blog/2009-06-11-cryptographic-right-answers.html

## <font color='blue'>_Checkpoint 1_:</font> <font color='gray'>encryption issues</font>

**DO NOT** continue further in the notebook, wait for the tutor
to tell you :-). Don't spoil the seminar for yourself.

## Task 2: <font color='gray'>How to DO it</font>

### Issues #1 and #2: Key generation

To generate a secure key we need to use bigger key sizes and follow [**the recommendations**](https://cryptography.io/en/latest/hazmat/primitives/asymmetric/rsa/?highlight=verify#cryptography.hazmat.primitives.asymmetric.rsa.generate_private_key)<br>
for public exponent.

In [ ]:
from cryptography.hazmat.primitives import hashes

# generate a 4096-bit long private key
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=4096,
)
print(isinstance(private_key, rsa.RSAPrivateKey))

### Issues #3: Message (un)padding
Again, we need to follow [**the recommendations**](https://cryptography.io/en/latest/hazmat/primitives/asymmetric/rsa/?highlight=verify#cryptography.hazmat.primitives.asymmetric.padding.OAEP) and use different padding scheme - OAEP.

In [ ]:
pad = padding.OAEP(
    mgf=padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None
)

### Issue #4: Encryption and decryption

- [`encrypt()`](https://cryptography.io/en/latest/hazmat/primitives/asymmetric/rsa/?highlight=verify#cryptography.hazmat.primitives.asymmetric.rsa.RSAPublicKey.encrypt) documentation
- [`.decrypt()`](https://cryptography.io/en/latest/hazmat/primitives/asymmetric/rsa/?highlight=verify#cryptography.hazmat.primitives.asymmetric.rsa.RSAPrivateKey.decrypt) documentation

In [ ]:
# public key is accesible through the private part
public_key = private_key.public_key()
print(isinstance(public_key, rsa.RSAPublicKey))

# We can encrypt only short messages
# to encrypt we use the public key
message = b"message"
ciphertext = public_key.encrypt(
    plaintext=message,
    padding=pad,
)

# to decrypt we use the private key
plaintext = private_key.decrypt(
    ciphertext=ciphertext,
    padding=pad,
)
print(plaintext == message)

## Task 3: <font color='gray'>Implement RSA encrypt/decrypt functions</font>

Consult the [`cryptography.rsa`](https://cryptography.io/en/latest/hazmat/primitives/asymmetric/rsa/) module if necessary :)

In [ ]:
from typing import Optional


def rsa_encrypt(pubkey: rsa.RSAPublicKey, message: bytes) -> bytes:
    # TODO: define the padding
    # TODO: try encrypting the message
    # TODO: return the ciphertext
    pass


def rsa_decrypt(privkey: rsa.RSAPrivateKey, ciphertext: bytes) -> Optional[bytes]:
    # TODO: define the same padding
    # TODO: try decrypting the message
    # TODO: return the ciphertext
    pass

In [ ]:
def rsa_encrypt(pubkey: rsa.RSAPublicKey, message: bytes) -> Optional[bytes]:
    # TODO: define the padding
    padding = padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None,
    )
    # TODO: encrypt the message
    try:
        ciphertext = public_key.encrypt(
            message,
            padding,
        )
    except:
        ciphertext = None
    # TODO: return the ciphertext
    return ciphertext


def rsa_decrypt(privkey: rsa.RSAPrivateKey, ciphertext: bytes) -> Optional[bytes]:
    padding = padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None,
    )
    try:
        plaintext = private_key.decrypt(
            ciphertext,
            padding,
        )
    except:
        plaintext = None

    return plaintext

## <font color='blue'>_Checkpoint 2:_</font> <font color='gray'>webserver key API</font>

**DO NOT** continue futher in the notebook, wait for the tutor to tell you :-).<br>

Don't spoil the seminar for yourself.

## Task 4: <font color='gray'>Communication using RSA</font>

Contrary to symmetric ciphers we **do not need** a second channel
in order to share the secret key. However, we need to publish
our public key. Next to the previous `send_message`, `recv_message` functions
we have prepared also functions to publish your own key and to
fetch someone elses key:

```python
send_message(uco_from: int, uco_to: int, content: bytes) -> str

recv_message(uco: int) -> Mapping[str, Union[int, bytes]]

publish_key(uco: int, key: rsa.RSAPublicKey) -> str

fetch_key(uco: int) -> Optional[rsa.RSAPublicKey]
```

1. **Generate** your own private/public RSA keypair.
2. **Publish** your public key.
3. **Fetch** a friend's public key.
4. **Encrypt** a message for a friend and **send** it.
5. **Receive** a message from a friend and **decrypt** it.

In [ ]:
from server_communication import send_message, recv_message
from server_communication import publish_key, fetch_key

In [ ]:
help(recv_message)

In [ ]:
help(fetch_key)

In [ ]:
# TODO: 1. **Generate** your bown private/public RSA keypair.
# TODO: 2. **Publish** your public key.
# TODO: 3. **Fetch** a friend's public key.
# TODO: 4. **Encrypt** a message for a friend and **send** it.
# TODO: 5. **Receive** a message from a friend and **decrypt** it.

## (Bonus) Task: <font color='gray'>Walkthrough "textbook" RSA</font>

When we use the RSA API from the `cryptography` module we
do not directly handle the primes $p,q$ and the private exponent
$d$. However, even if hidden, those values are still present.

### Refresh the math behind RSA encryption/decryption routines

RSA consists of **private** primes $p,q$ that form the modulus

\begin{align}\label{N}
N= p\cdot q
\end{align}

The **encryption** of a message $m$ is then defined as:

\begin{align*}
c=m^e \mod N
\end{align*}

where $e$ is the public exponent (usually $65537$) and $c$ is
the ciphertext. The **decryption** is done using the **private**
exponent $d$ that satisfies:

\begin{align}\label{priv_def}
e\cdot d \equiv 1 \mod \phi(N)
\end{align}

For $p,q$ primes it holds that

\begin{align*}
\phi(N) = \phi(p \cdot q) = \phi(p) \cdot \phi(q) = (p-1)(q-1)
\end{align*}

The **decryption** routine than looks like this

\begin{align}\label{decryption}
c^d = (m^e)^d \equiv m^{e\cdot d} \equiv m \mod N
\end{align}

Use the values before to verify the equalities/congruencies
(1), (2) and (3).

In [ ]:
p = private_key.private_numbers().p
q = private_key.private_numbers().q
d = private_key.private_numbers().d

e = private_key.public_key().public_numbers().e
N = private_key.public_key().public_numbers().n

# we will generate a random message
import random

m = random.randint(0, N - 1)

# HINT: use the pow(base, exponent, modulus) to perform
# modular exponentiation quickly

# TODO: equation (1)

# TODO: equation (2)

# TODO: equation (3)

In [ ]:
# FIXME: removing this solution cell

# check the modulus
print(p * q == N)

# check the private key
print((d * e) % ((p - 1) * (q - 1)) == 1)

# The following might take a loooooooong time to calculate..
# print(((m**e) ** d) % N == 1 )
# faster implementation
ct = pow(m, e, N)
pt = pow(ct, d, N)
print(pt == m)